In [1]:
import tensorflow as tf
from tensorflow.keras import models, layers
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

In [2]:
IMG_SIZE = 256
BATCH_SIZE = 32
CHANNELS = 3
EPOCH = 10

In [3]:
dataset = tf.keras.preprocessing.image_dataset_from_directory(
    "breast_cancer",
    shuffle= True,
    image_size= (IMG_SIZE, IMG_SIZE),
    batch_size= BATCH_SIZE
)

Found 7926 files belonging to 2 classes.


In [4]:
class_names = dataset.class_names
class_names

['benign', 'malignant']

In [5]:
len(dataset)

248

training data = 80% ;
validation data = 10% ;
testing data = 10%

In [6]:
def dataset_partition(ds, train_split=0.8, val_split=0.1, test_split=0.1, shuffle=True, shuffle_size=10000):
    
    ds_size = len(ds)
    
    if shuffle:
        ds = ds.shuffle(shuffle_size, seed=12)
        
    train_size = int(train_split * ds_size)
    val_size = int(val_split * ds_size)
    
    train_ds = ds.take(train_size)
    val_ds = ds.skip(train_size).take(val_size)
    test_ds = ds.skip(train_size).skip(val_size)
    
    return train_ds, val_ds, test_ds

In [7]:
train_ds, val_ds, test_ds = dataset_partition(dataset)

In [8]:
len(train_ds)

198

In [9]:
len(val_ds)

24

In [10]:
len(test_ds)

26

In [11]:
train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)
val_ds = val_ds.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)
test_ds = test_ds.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)

In [12]:
resize_and_rescale = tf.keras.Sequential([
    layers.experimental.preprocessing.Resizing(IMG_SIZE, IMG_SIZE),
    layers.experimental.preprocessing.Rescaling(1.0/255)
])

data_augmentation = tf.keras.Sequential([
    layers.experimental.preprocessing.RandomFlip("horizontal_and_vertical"),
    layers.experimental.preprocessing.RandomRotation(0.2)
])

In [13]:
input_shape = (BATCH_SIZE, IMG_SIZE, IMG_SIZE, CHANNELS)
n_classes = 2

model = models.Sequential([
    resize_and_rescale,
    #data_augmentation,
    
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=input_shape),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, kernel_size= (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, kernel_size= (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(n_classes, activation='softmax')
])

model.build(input_shape=input_shape)

In [14]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 sequential (Sequential)     (32, 256, 256, 3)         0         
                                                                 
 conv2d (Conv2D)             (32, 254, 254, 32)        896       
                                                                 
 max_pooling2d (MaxPooling2D  (32, 127, 127, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (32, 125, 125, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (32, 62, 62, 64)         0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (32, 60, 60, 64)         

In [15]:
model.compile(
    optimizer='adam',
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
    metrics=['accuracy']
)

In [16]:
history = model.fit(
    train_ds,
    epochs=EPOCH,
    batch_size=BATCH_SIZE,
    verbose=1,
    validation_data=val_ds
)

Epoch 1/10
198/198 [==============================] - 374s 2s/step - loss: 0.5025 - accuracy: 0.7779 - val_loss: 0.5502 - val_accuracy: 0.7461
Epoch 2/10
198/198 [==============================] - 259s 1s/step - loss: 0.4462 - accuracy: 0.8220 - val_loss: 0.4272 - val_accuracy: 0.8516
Epoch 3/10
198/198 [==============================] - 263s 1s/step - loss: 0.4247 - accuracy: 0.8362 - val_loss: 0.3879 - val_accuracy: 0.8568
Epoch 4/10
198/198 [==============================] - 267s 1s/step - loss: 0.4128 - accuracy: 0.8377 - val_loss: 0.5007 - val_accuracy: 0.7669
Epoch 5/10
198/198 [==============================] - 274s 1s/step - loss: 0.3899 - accuracy: 0.8508 - val_loss: 0.3922 - val_accuracy: 0.8542
Epoch 6/10
198/198 [==============================] - 273s 1s/step - loss: 0.3856 - accuracy: 0.8527 - val_loss: 0.3699 - val_accuracy: 0.8607
Epoch 7/10
198/198 [==============================] - 272s 1s/step - loss: 0.3699 - accuracy: 0.8598 - val_loss: 0.3735 - val_accuracy: 0.8620

In [17]:
score = model.evaluate(test_ds)

26/26 [==============================] - 70s 339ms/step - loss: 0.3209 - accuracy: 0.8798


In [18]:
score

[0.3208509683609009, 0.879807710647583]

In [20]:
model_version = 1
model.save(f"model/{model_version}")

INFO:tensorflow:Assets written to: model/1\assets


INFO:tensorflow:Assets written to: model/1\assets
